In [4]:
import os

In [5]:
os.chdir('../')

In [6]:
%pwd

'c:\\Users\\mayuo\\OneDrive\\Documents\\Machine Learning by Abraham\\MLOps wine'

#### code the entity first 

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    C: int
    solver: str
    target_column: str

#### config manager

In [8]:
from src.MLOps_wine.constants import *
from src.MLOps_wine.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.LogisticRegression
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            C = params.C,
            solver = params.solver,
            target_column = schema.name
            
        )

        return model_trainer_config

#### write components

In [15]:
import pandas as pd
import os
from src.MLOps_wine import logger
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, classification_report
import joblib

In [16]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # oe = OrdinalEncoder()
        # train_data[self.config.target_column] = oe.fit_transform(train_data[[self.config.target_column]])
        # test_data[self.config.target_column] = oe.transform(test_data[[self.config.target_column]])
        label_mapping = {'low': 0, 'medium': 1, 'high': 2}
        train_data[self.config.target_column] = train_data[self.config.target_column].map(label_mapping)
        test_data[self.config.target_column] = test_data[self.config.target_column].map(label_mapping)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]


        lr = LogisticRegression(C=self.config.C,solver=self.config.solver, random_state=42)
        lr.fit(train_x, train_y)

        train_predictions = lr.predict(train_x)

        # Calculate accuracy using accuracy_score
        accuracy = accuracy_score(train_y, train_predictions)
        report = classification_report(train_y, train_predictions, output_dict=True)

         # Print metrics
        print(f"Training Accuracy: {accuracy:.4f}")
        print(f"Training Precision: {report['accuracy']:.4f}")  # Access accuracy from report
        print(f"Training Recall: {report['weighted avg']['recall']:.4f}")
        print(f"Training F1-Score: {report['weighted avg']['f1-score']:.4f}")
        print(train_y.value_counts(), test_y.value_counts())

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        #print(train_y.value_counts(), test_y.value_counts())

#### The pipeline

In [17]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-04-30 12:23:21,344: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-30 12:23:21,346: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-30 12:23:21,351: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-04-30 12:23:21,354: INFO: common: created directory at: artifacts]
[2024-04-30 12:23:21,356: INFO: common: created directory at: artifacts/model_trainer]
Training Accuracy: 0.8131
Training Precision: 0.8131
Training Recall: 0.8131
Training F1-Score: 0.7293
quality
1    1040
2     192
0      47
Name: count, dtype: int64 quality
1    281
2     24
0     15
Name: count, dtype: int64


c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\MLOps wine\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\MLOps wine\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mayuo\OneDrive\Documents\Machine Learning by Abraham\MLOps wine\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `ze